# Data Challenge: Cargo Volume
- We are tasked to predict cargo colume through San Francisco International Airport

In [1]:
import pandas as pd

train_df = pd.read_csv('Cargo Volume - Train.csv')
test_df = pd.read_csv('Cargo Volume - Test.csv')
train_df.head()

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Metric TONS
0,201601,Sun Country Airlines,SY,Sun Country Airlines,SY,Domestic,US,Enplaned,Cargo,Passenger,8.871509
1,200811,United Airlines,UA,United Airlines,UA,Domestic,US,Deplaned,Cargo,Passenger,53.946194
2,200709,Qantas Airways,QF,Qantas Airways,QF,International,Australia / Oceania,Deplaned,Express,Passenger,0.043999
3,201608,Philippine Airlines,PR,Philippine Airlines,PR,International,Asia,Deplaned,Mail,Passenger,19.023530
4,201708,Singapore Airlines,SQ,Singapore Airlines,SQ,International,Asia,Deplaned,Express,Passenger,6.951420


In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27230 entries, 0 to 27229
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Activity Period              27230 non-null  int64  
 1   Operating Airline            27230 non-null  object 
 2   Operating Airline IATA Code  27199 non-null  object 
 3   Published Airline            27230 non-null  object 
 4   Published Airline IATA Code  27199 non-null  object 
 5   GEO Summary                  27230 non-null  object 
 6   GEO Region                   27230 non-null  object 
 7   Activity Type Code           27230 non-null  object 
 8   Cargo Type Code              27230 non-null  object 
 9   Cargo Aircraft Type          27230 non-null  object 
 10  Cargo Metric TONS            27230 non-null  float64
dtypes: float64(1), int64(1), object(9)
memory usage: 2.3+ MB


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6807 entries, 0 to 6806
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Activity Period              6807 non-null   int64 
 1   Operating Airline            6807 non-null   object
 2   Operating Airline IATA Code  6793 non-null   object
 3   Published Airline            6807 non-null   object
 4   Published Airline IATA Code  6793 non-null   object
 5   GEO Summary                  6807 non-null   object
 6   GEO Region                   6807 non-null   object
 7   Activity Type Code           6807 non-null   object
 8   Cargo Type Code              6807 non-null   object
 9   Cargo Aircraft Type          6807 non-null   object
dtypes: int64(1), object(9)
memory usage: 531.9+ KB


# Data Analysis
- We can see the majority of our dataset consists of Categorical data thus some type of encoding must take place.
- It should also be noted that the column 'Activity Period' is in a format that will eventually involve some Feature Engineering.
- From the project spec we expected the target column 'Cargo Metric TONS' to be missing from our test data set.

In [4]:
train_df.isna().sum()

Activity Period                 0
Operating Airline               0
Operating Airline IATA Code    31
Published Airline               0
Published Airline IATA Code    31
GEO Summary                     0
GEO Region                      0
Activity Type Code              0
Cargo Type Code                 0
Cargo Aircraft Type             0
Cargo Metric TONS               0
dtype: int64

In [5]:
train_df = train_df.dropna()

In [6]:
test_df.isna().sum()

Activity Period                 0
Operating Airline               0
Operating Airline IATA Code    14
Published Airline               0
Published Airline IATA Code    14
GEO Summary                     0
GEO Region                      0
Activity Type Code              0
Cargo Type Code                 0
Cargo Aircraft Type             0
dtype: int64

In [7]:
test_df = test_df.dropna()

In [8]:
train_df.describe(include='object')

,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type
count,27199,27199,27199,27199,27199,27199,27199,27199,27199
unique,89,87,82,80,2,9,2,3,3
top,United Airlines,UA,United Airlines,UA,International,Asia,Deplaned,Cargo,Passenger
freq,2551,4565,2557,4578,19849,10077,13916,14877,22677


In [9]:
train_df.nunique()

Activity Period                  186
Operating Airline                 89
Operating Airline IATA Code       87
Published Airline                 82
Published Airline IATA Code       80
GEO Summary                        2
GEO Region                         9
Activity Type Code                 2
Cargo Type Code                    3
Cargo Aircraft Type                3
Cargo Metric TONS              24872
dtype: int64

# Feature Engineering: Activity Period
- There are monthly rhythms for all air travel, thus it would benefit us to utilize this data.
- Sliceing the yyyymm formate into separate columns should increase model accuracy.
- We need to remember to implement these transformations on our test dataframe.

In [10]:
train_df['Year'] = train_df['Activity Period'].astype(str).str.slice(0, 4).astype(int)
train_df['Month'] = train_df['Activity Period'].astype(str).str.slice(4, 6).astype(int)

train_df.columns

Index(['Activity Period', 'Operating Airline', 'Operating Airline IATA Code',
       'Published Airline', 'Published Airline IATA Code', 'GEO Summary',
       'GEO Region', 'Activity Type Code', 'Cargo Type Code',
       'Cargo Aircraft Type', 'Cargo Metric TONS', 'Year', 'Month'],
      dtype='object')

In [11]:
test_df['Year'] = test_df['Activity Period'].astype(str).str.slice(0, 4).astype(int)
test_df['Month'] = test_df['Activity Period'].astype(str).str.slice(4, 6).astype(int)

test_df.columns

Index(['Activity Period', 'Operating Airline', 'Operating Airline IATA Code',
       'Published Airline', 'Published Airline IATA Code', 'GEO Summary',
       'GEO Region', 'Activity Type Code', 'Cargo Type Code',
       'Cargo Aircraft Type', 'Year', 'Month'],
      dtype='object')

# Feature Selection
- Since our values are nominal, label encoding would not be an ideal way to encode our data.
- The usual way to encode nominal variables is with a one-hot-encoding.
- However, the columns with large amounts of unique values could introduce the curse of dimensionality.
- Because of this I'd like to eliminate redundacy

In [12]:
print(train_df['Operating Airline'].unique())
print()
print(train_df['Published Airline'].unique())

['Sun Country Airlines' 'United Airlines' 'Qantas Airways'
 'Philippine Airlines' 'Singapore Airlines' 'Nippon Cargo Airlines'
 'Lufthansa German Airlines' 'COPA Airlines, Inc.' 'Alaska Airlines'
 'Asiana Airlines' 'Cathay Pacific' 'Korean Air Lines'
 'All Nippon Airways' 'Southwest Airlines' 'Air France' 'EVA Airways'
 'United Airlines - Pre 07/01/2013' 'Northwest Airlines (became Delta)'
 'Aer Lingus' 'TACA International Airlines, S.A.' 'China Southern'
 'Air New Zealand' 'Air China' 'SAS Airlines' 'China Eastern'
 'Frontier Airlines' 'Horizon Air' 'Emirates' 'JetBlue Airways'
 'Kalitta Air' 'Japan Airlines' 'Delta Air Lines' 'French Bee'
 'Air Canada' 'Southern Air' 'Compass Airlines' 'British Airways'
 'Redding Aero Enterprises' 'Swiss International' 'Etihad Airways'
 'KLM Royal Dutch Airlines' 'China Airlines' 'Hawaiian Airlines'
 'US Airways' 'American Airlines' 'Virgin Atlantic' 'Federal Express'
 'Ameriflight' 'SkyWest Airlines' 'LAN Peru' 'Midwest Airlines' 'ABX Air'
 'Air Ind

In [13]:
from sklearn.preprocessing import LabelEncoder

# Encoding so that we can check the correlation
le_operating = LabelEncoder()
le_published = LabelEncoder()

train_df['Operating Airline Encoded'] = le_operating.fit_transform(train_df['Operating Airline'])
train_df['Published Airline Encoded'] = le_published.fit_transform(train_df['Published Airline'])

# Apply Spearman's correlation method
operating_corr = train_df['Operating Airline Encoded'].corr(train_df['Cargo Metric TONS'], method='spearman')
published_corr = train_df['Published Airline Encoded'].corr(train_df['Cargo Metric TONS'], method='spearman')

print(f"Correlation with Cargo Metric TONS : Operating Airline: {operating_corr}")
print(f"Correlation with Cargo Metric TONS : Published Airline: {published_corr}")

train_df.drop(['Operating Airline Encoded', 'Published Airline Encoded'], axis=1, inplace=True)

Correlation with Cargo Metric TONS : Operating Airline: -0.04049956827473915
Correlation with Cargo Metric TONS : Published Airline: 0.0023500585583753303


In [14]:
print(train_df['Operating Airline IATA Code'].unique())
print()
print(train_df['Published Airline IATA Code'].unique())

['SY' 'UA' 'QF' 'PR' 'SQ' 'KZ' 'LH' 'CM' 'AS' 'OZ' 'CX' 'KE' 'NH' 'WN'
 'AF' 'BR' 'NW' 'EI' 'TA' 'CZ' 'NZ' 'CA' 'SK' 'MU' 'F9' 'QX' 'EK' 'B6'
 'K4' 'JL' 'DL' 'BF' 'AC' '9S' 'CP' 'BA' 'bxr' 'LX' 'EY' 'KL' 'CI' 'HA'
 'US' 'AA' 'VS' 'FX' 'A8' 'OO' 'LP' 'YX' 'GB' 'AI' 'WI' 'FI' 'AB' 'IB'
 'TZ' 'DY' 'FJ' '5Y' 'ER' 'TK' 'CK' '8C' 'CV' 'KR' '2Q' 'DI' 'MT' 'HX'
 'MQ' 'AY' 'TP' 'WW' '5X' 'WO' '9W' 'EZ' 'IG' 'RW' 'LY' 'VX' 'QK' 'AM'
 'YV' 'QR' 'XJ']

['SY' 'UA' 'QF' 'PR' 'SQ' 'KZ' 'LH' 'CM' 'AS' 'OZ' 'CX' 'KE' 'NH' 'WN'
 'AF' 'BR' 'NW' 'EI' 'TA' 'CZ' 'NZ' 'CA' 'SK' 'MU' 'F9' 'EK' 'B6' 'K4'
 'JL' 'DL' 'BF' 'AC' '9S' 'AA' 'BA' 'bxr' 'LX' 'EY' 'KL' 'CI' 'HA' 'US'
 'VS' 'FX' 'A8' 'LP' 'YX' 'GB' 'AI' 'WI' 'FI' 'AB' 'IB' 'TZ' 'DY' 'FJ'
 '5Y' 'ER' 'TK' 'CK' '8C' 'CV' 'KR' '2Q' 'DI' 'MT' 'HX' 'AY' 'TP' 'WW'
 '5X' 'WO' '9W' 'EZ' 'IG' 'RW' 'LY' 'VX' 'AM' 'QR']


In [15]:
from sklearn.preprocessing import LabelEncoder

# Encoding so that I can check the correlation
le_operating = LabelEncoder()
le_published = LabelEncoder()

train_df['Operating Airline IATA Code Encoded'] = le_operating.fit_transform(train_df['Operating Airline IATA Code'])
train_df['Published Airline IATA Code Encoded'] = le_published.fit_transform(train_df['Published Airline IATA Code'])

# Apply Spearman's correlation method 
operating_corr = train_df['Operating Airline IATA Code Encoded'].corr(train_df['Cargo Metric TONS'], method='spearman')
published_corr = train_df['Published Airline IATA Code Encoded'].corr(train_df['Cargo Metric TONS'], method='spearman')

print(f"Correlation with Cargo Metric TONS : Operating Airline IATA Code: {operating_corr}")
print(f"Correlation with Cargo Metric TONS : Published Airline IATA Code: {published_corr}")

train_df.drop(['Operating Airline IATA Code Encoded', 'Published Airline IATA Code Encoded'], axis=1, inplace=True)

Correlation with Cargo Metric TONS : Operating Airline IATA Code: -0.03603762157166911
Correlation with Cargo Metric TONS : Published Airline IATA Code: 0.01943558085210896


# Findings
- All four columns from the dataset show very weak correlations to the target.
- With this information I'll remove the weakest correlations in an effort to reduce dimensonality.

In [16]:
train_df.drop(['Published Airline', 'Published Airline IATA Code', 'Activity Period'], axis=1, inplace=True)
test_df.drop(['Published Airline', 'Published Airline IATA Code', 'Activity Period'], axis=1, inplace=True)

In [17]:
train_df.columns

Index(['Operating Airline', 'Operating Airline IATA Code', 'GEO Summary',
       'GEO Region', 'Activity Type Code', 'Cargo Type Code',
       'Cargo Aircraft Type', 'Cargo Metric TONS', 'Year', 'Month'],
      dtype='object')

In [18]:
test_df.columns

Index(['Operating Airline', 'Operating Airline IATA Code', 'GEO Summary',
       'GEO Region', 'Activity Type Code', 'Cargo Type Code',
       'Cargo Aircraft Type', 'Year', 'Month'],
      dtype='object')

# Feature Engineering: Month
- To improve the modeling process we'll augment the Month column into sine and cosine.
- This will highlight any cyclical patterns in the dataset

In [19]:
import numpy as np

train_df['Month_sin'] = np.sin(train_df.Month * 2 * np.pi / len(train_df.Month.unique()))
train_df['Month_cos'] = np.cos(train_df.Month * 2 * np.pi / len(train_df.Month.unique()))

cols = train_df.columns.tolist()
cols.remove('Cargo Metric TONS')
cols.append('Cargo Metric TONS')

train_df = train_df[cols]

train_df.drop(['Month'], axis=1, inplace=True)
train_df.head()

,Operating Airline,Operating Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Year,Month_sin,Month_cos,Cargo Metric TONS
0,Sun Country Airlines,SY,Domestic,US,Enplaned,Cargo,Passenger,2016,0.500000,8.660254e-01,8.871509
1,United Airlines,UA,Domestic,US,Deplaned,Cargo,Passenger,2008,-0.500000,8.660254e-01,53.946194
2,Qantas Airways,QF,International,Australia / Oceania,Deplaned,Express,Passenger,2007,-1.000000,-1.836970e-16,0.043999
3,Philippine Airlines,PR,International,Asia,Deplaned,Mail,Passenger,2016,-0.866025,-5.000000e-01,19.023530
4,Singapore Airlines,SQ,International,Asia,Deplaned,Express,Passenger,2017,-0.866025,-5.000000e-01,6.951420


In [20]:
test_df['Month_sin'] = np.sin(test_df.Month * 2 * np.pi / len(test_df.Month.unique()))
test_df['Month_cos'] = np.cos(test_df.Month * 2 * np.pi / len(test_df.Month.unique()))

test_df.drop(['Month'], axis=1, inplace=True)
test_df.head()

,Operating Airline,Operating Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Year,Month_sin,Month_cos
0,United Airlines,UA,Domestic,US,Deplaned,Mail,Passenger,2009,-0.5,-8.660254e-01
1,Nippon Cargo Airlines,KZ,International,Asia,Deplaned,Cargo,Freighter,2020,0.5,8.660254e-01
2,China Airlines,CI,International,Asia,Enplaned,Cargo,Passenger,2011,-0.5,-8.660254e-01
3,China Airlines,CI,International,Asia,Deplaned,Cargo,Passenger,2018,-1.0,-1.836970e-16
4,China Cargo Airlines,CK,International,Asia,Enplaned,Cargo,Freighter,2008,-0.5,8.660254e-01


# Ecoding
- As previously mentioned we are working with categorical values with no intrinsic ordering.
- Thus we will one-hot-encode the strings in our dataset.

In [21]:
from sklearn.preprocessing import LabelEncoder

label_enc = {}
cols_to_enc = ['Operating Airline', 'Operating Airline IATA Code', 'GEO Summary', 
               'GEO Region', 'Activity Type Code', 'Cargo Type Code', 'Cargo Aircraft Type']

label_train_df = train_df.copy()
label_test_df = test_df.copy()

for col in cols_to_enc:
    label_enc[col] = LabelEncoder()
    
    combined_data = pd.concat([train_df[col], test_df[col]], axis=0).astype(str)
    label_enc[col].fit(combined_data)
    label_train_df[col] = label_enc[col].transform(train_df[col].astype(str))
    
    label_test_df[col] = label_enc[col].transform(test_df[col].astype(str))

In [22]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse_output=False)

combined_data = pd.concat([label_train_df[cols_to_enc], label_test_df[cols_to_enc]], axis=0)
one_hot_encoder.fit(combined_data)

one_hot_encoded_train = one_hot_encoder.transform(label_train_df[cols_to_enc])
encoded_train_df = pd.DataFrame(one_hot_encoded_train,
                                columns=one_hot_encoder.get_feature_names_out(cols_to_enc),
                                index=label_train_df.index)

one_hot_encoded_test = one_hot_encoder.transform(label_test_df[cols_to_enc])
encoded_test_df = pd.DataFrame(one_hot_encoded_test,
                               columns=one_hot_encoder.get_feature_names_out(cols_to_enc),
                               index=label_test_df.index)

encoded_train_df = pd.concat([label_train_df.drop(cols_to_enc, axis=1), encoded_train_df], axis=1)
encoded_test_df = pd.concat([label_test_df.drop(cols_to_enc, axis=1), encoded_test_df], axis=1)

# LinearRegression
- Linear Regression is dependent on scale.
- This means our one-hot-encoded values could cause issues.
- I'll use the label-encoded DataFrame to train.

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()

X = label_train_df.drop('Cargo Metric TONS', axis=1)
y = label_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lin_reg.fit(X_train, y_train)
lin_mse = mean_squared_error(y_test, lin_reg.predict(X_test))
print(f'LinearRegression RMSE: {np.sqrt(lin_mse)}')

LinearRegression RMSE: 383.356598659381


# Lasso
- Lasso uses the alpha term to prevent overfitting.
- Thus using the one-hot-encoded values should cause adverse effects.

In [24]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso(alpha=1)

X = encoded_train_df.drop('Cargo Metric TONS', axis=1)
y = encoded_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lasso.fit(X_train, y_train)
lasso_mse = mean_squared_error(y_test, lasso.predict(X_test))
print(f'Lasso RMSE: {np.sqrt(lasso_mse)}')

Lasso RMSE: 318.9516694422963


# Ridge
- Ridge is another regression model that uses alpha to prevent overfitting.

In [25]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1)

X = encoded_train_df.drop('Cargo Metric TONS', axis=1)
y = encoded_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

ridge.fit(X_train, y_train)
ridge_mse = mean_squared_error(y_test, ridge.predict(X_test))
print(f'Ridge RMSE: {np.sqrt(ridge_mse)}')

Ridge RMSE: 331.8139156095431


# CART
- CART decision trees handle outlines well.
- This has been one of my most consistent models.

In [26]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor(criterion='friedman_mse',
                               max_depth=10,
                               min_samples_leaf=2,
                               min_samples_split=15)

X = encoded_train_df.drop('Cargo Metric TONS', axis=1)
y = encoded_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dt_reg.fit(X_train, y_train)

dt_mse = mean_squared_error(y_test, dt_reg.predict(X_test))
print(f'CART RMSE: {np.sqrt(dt_mse)}')

CART RMSE: 190.45832351973925


# XGBoost
- Because we one-hot-encoded two columns xgb is an ideal model beacuse of how well it works on large ds.
- Since this Data Challenge is one envolving a regression, xgb's gradient boosting framework is ideal.

In [27]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor(colsample_bytree = 0.814, 
                          learning_rate = 0.049,
                          max_depth = 9,  
                          n_estimators = 293,
                          n_jobs=-1,
                          reg_lambda=3,
                          reg_alpha=1)

X = encoded_train_df.drop('Cargo Metric TONS', axis=1)
y = encoded_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

xg_reg.fit(X_train, y_train)
xg_mse = mean_squared_error(y_test, xg_reg.predict(X_test))
print(f'XGBoost RMSE: {np.sqrt(xg_mse)}')

XGBoost RMSE: 193.4169174859524


# RandomForest

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(max_depth=30,
                               n_estimators=50,
                               n_jobs=-1,
                               min_samples_leaf=4)

X = encoded_train_df.drop('Cargo Metric TONS', axis=1)
y = encoded_train_df['Cargo Metric TONS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf_reg.fit(X_train, y_train)
rf_mse = mean_squared_error(y_test, rf_reg.predict(X_test))
print(f'Random Forest RMSE: {np.sqrt(rf_mse)}')

Random Forest RMSE: 167.24325656258398


# Choosing a model : RandomForest
- Since hyperparameter tuning RandomForest and XGBoost they have produced very similar RMSE scores.
- I was going to use k-fold to compare the performace of the two one last time but have decided against it.
- LinearRegression, Lasso, and Ridge are all linear models.
- Because all three of those models preformed the worst it suggest that either non-linearity or high dimensionality came into play.
- As for why RandomForest bested XGBoost I'm not sure.
- RandomForest's may have pickup up on the seasonal features of the engineered Month columns better than XGBoost.

In [30]:
X = encoded_test_df
y_target_col = rf_reg.predict(X)
y_target_col_df = pd.DataFrame({'Cargo Metric TONS': y_target_col})
y_target_col_df.to_csv('dc1.csv', index=False)